<a href="https://colab.research.google.com/github/AgrawalShiven/Cross_camera_tracking-YOLO/blob/main/Cross_Camera_Player_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

  Attempting uninstall: nvidia-cuda-runtime-cu12
    Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
    Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
  Attempting uninstall: nvidia-cuda-nvrtc-cu12
    Found existing installation: nvidia-cuda-nvrtc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvrtc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvrtc-cu12-12.5.82
  Attempting uninstall: nvidia-cuda-cupti-cu12
    Found existing installation: nvidia-cuda-cupti-cu12 12.5.82
    Uninstalling nvidia-cuda-cupti-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-cupti-cu12-12.5.82
  Attempting uninstall: nvidia-cublas-cu12
    Found existing installation: nvidia-cublas-cu12 12.5.3.2
    Uninstalling nvidia-cublas-cu12-12.5.3.2:
      Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Unins

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
from skimage.feature import hog

model = YOLO('best.pt')
model.to('cuda')
video_1 = 'broadcast.mp4'
video_2 = 'tacticam.mp4'

from skimage.feature import hog
import cv2
import numpy as np

def extract_hist_hog(frame, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    player_crop = frame[y1:y2, x1:x2]

    if player_crop.size == 0 or x2 <= x1 or y2 <= y1:
        return np.zeros(512 + 3780)
    resized_crop = cv2.resize(player_crop, (64, 128))

    hist = cv2.calcHist([resized_crop], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()

    gray_crop = cv2.cvtColor(resized_crop, cv2.COLOR_BGR2GRAY)
    hog_feat = hog(
        gray_crop,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        transform_sqrt=True,
        feature_vector=True
    )

    combined_feat = np.concatenate([hist, hog_feat])
    return combined_feat


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def run_tracking(video_path, model, tracker_type='botsort.yaml', show_video=False, save_video=True, output_name='output_with_ids.mp4'):
    results = model.track(
        source=video_path,
        tracker=tracker_type,
        persist=True,
        verbose=False,
        stream=True
    )

    track_data = []
    cap = cv2.VideoCapture(video_path)
    frame_id = 0

    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(
            output_name, fourcc,
            cap.get(cv2.CAP_PROP_FPS),
            (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        )

    for result in results:
        ret, frame = cap.read()
        if not ret:
            break
        frame_track = []
        for box in result.boxes:
            if int(box.cls[0]) == 2:  # Use 2 for 'player' class
                if box.id is None:
                    continue
                track_id = int(box.id[0])
                bbox = list(map(int, box.xyxy[0]))
                feature = extract_hist_hog(frame, bbox)
                frame_track.append({
                    'track_id': track_id,
                    'bbox': bbox,
                    'hist': feature,
                    'frame_id': frame_id
                })

                # Draw bounding box and ID
                x1, y1, x2, y2 = bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'ID: {track_id}', (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if show_video:
            from google.colab.patches import cv2_imshow
            cv2_imshow(frame)

        if save_video:
            out.write(frame)

        track_data.append(frame_track)
        frame_id += 1

    cap.release()
    if save_video:
        out.release()
    return track_data


In [4]:
def build_track_features(track_data):
    track_features = {}
    for frame_tracks in track_data:
        for obj in frame_tracks:
            tid = obj['track_id']
            if tid not in track_features:
                track_features[tid] = []
            track_features[tid].append(obj['hist'])
    for tid in track_features:
        track_features[tid] = np.mean(track_features[tid], axis=0)
    return track_features


In [5]:
def match_tracks(features1, features2, threshold=0.7):
    mapping = {}
    for tid2, feat2 in features2.items():
        best_tid1 = None
        best_score = -1
        for tid1, feat1 in features1.items():
            score = np.dot(feat1, feat2) / (np.linalg.norm(feat1) * np.linalg.norm(feat2) + 1e-6)
            if score > best_score:
                best_score = score
                best_tid1 = tid1
        if best_score > threshold:
            mapping[tid2] = best_tid1
    return mapping


In [10]:
if __name__ == '__main__':
    model = YOLO('best.pt')

    print("Tracking broadcast video...")
    broadcast_tracks = run_tracking(
        'broadcast.mp4',
        model,
        tracker_type='botsort.yaml',
        output_name='broadcast_with_ids.mp4'
    )

    print("Tracking tacticam video...")
    tacticam_tracks = run_tracking(
        'tacticam.mp4',
        model,
        tracker_type='botsort.yaml',
        output_name='tacticam_with_ids.mp4'
    )

    broadcast_features = build_track_features(broadcast_tracks)
    tacticam_features = build_track_features(tacticam_tracks)

    print("Matching players across feeds...")
    player_id_mapping = match_tracks(broadcast_features, tacticam_features, threshold=0.7)

    print("Player ID mapping from tacticam to broadcast:")
    for tacticam_id, broadcast_id in player_id_mapping.items():
        print(f"Tacticam player {tacticam_id} -> Broadcast player {broadcast_id}")


Tracking broadcast video...
Tracking tacticam video...
Matching players across feeds...
Player ID mapping from tacticam to broadcast:
Tacticam player 180 -> Broadcast player 22
Tacticam player 181 -> Broadcast player 24
Tacticam player 182 -> Broadcast player 25
Tacticam player 183 -> Broadcast player 25
Tacticam player 184 -> Broadcast player 15
Tacticam player 185 -> Broadcast player 25
Tacticam player 186 -> Broadcast player 20
Tacticam player 187 -> Broadcast player 18
Tacticam player 188 -> Broadcast player 18
Tacticam player 189 -> Broadcast player 16
Tacticam player 190 -> Broadcast player 18
Tacticam player 191 -> Broadcast player 25
Tacticam player 192 -> Broadcast player 18
Tacticam player 193 -> Broadcast player 18
Tacticam player 194 -> Broadcast player 24
Tacticam player 195 -> Broadcast player 25
Tacticam player 196 -> Broadcast player 17
Tacticam player 197 -> Broadcast player 18
Tacticam player 198 -> Broadcast player 21
Tacticam player 199 -> Broadcast player 20
Tactic